<a href="https://colab.research.google.com/github/danielaterra/ifmg_topicos_SI/blob/main/p_median_capacited_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Problema das P-Medianas Capacitado
Usaremos aqui o arquivo do TSP-50 locais, sendo T, K parâmetros selecionados.
- Para bases do problema com parâmetros e melhor solução,consulte a OR-library: https://people.brunel.ac.uk/~mastjjb/jeb/orlib/pmedcapinfo.html

**Profa. Daniela Costa Terra**

In [330]:
import sys
import time
import numpy as np  # importa o pacote com 'alias' ou apelido de 'np'
import math
import matplotlib.pyplot as plt    # o módulo pyplot, parte do matplotlib, como plt
np.random.seed(1000)    ## produz o mesmos numeros 'pseudo'aleatórios a cada execução

In [259]:
# Baixe os 3 arquivos abaixo, disponíveis na tarefa, sem seguida faça um upload dos mesmos aqui.
from google.colab import files
uploaded = files.upload()    ## faça upload dos arquivos disponibilizados

Saving C50.TXT to C50 (2).TXT
Saving C50INFO.TXT to C50INFO (2).TXT
Saving 50-BestRoute.txt to 50-BestRoute (2).txt


In [271]:
def le_arq_vetor(arquivo, dtype = int):
    v = []
    with open(arquivo, "r") as arquivo:
      linhas = arquivo.readlines()
      #print(linhas)
    for linha in linhas:
        aux = linha.split(' ')
        for num in aux:
           if dtype == int:
               v.append(int(num))
           else:
               v.append(float(num))
    return np.array(v)


# Constroi matriz quadrada (simétrica) de distâncias entre cidades
# está  preenchida abaixo e acima da diagonal
def constroi_matriz_distancias(n, locals):
    matriz_dist = np.zeros((n,n))
    for i in range(n):
      cidi = (locals[i,1],locals[i,2])
      for j in range(i+1, n):
          cidj = (locals[j,1],locals[j,2])
          matriz_dist[i,j] = math.dist(cidi, cidj)
          matriz_dist[j,i] = matriz_dist[i,j]
    return matriz_dist


In [331]:
## Inicializações
MaxIter = 100   # número máximo de iterações para busca de soluções aleatórios

# Lê arquivos para o problema do caixeiro viajange
## Formato C..INFO.txt: n_cidades,  fo_solução_otima (menor distancia)
dados = le_arq_vetor("/content/C50INFO.TXT", dtype=float)     #arq de informações sobre o no. de cidades e solução ótima (em distancia)
n = int(dados[0])   # 51 locais


## Formado C50.txt: com 51 locais
# num_cidade   X   Y  (onde X,Y são as coordenadas de localização para cada cidade)
locals = le_arq_vetor("/content/C50.TXT", dtype = float)
locals = locals.reshape((n,3))

### Parêmetros para cálculo:
k = 5    # número de instalações
T = 10   # capacidade total de atendimento das instalações

# variáveis p/ cálculo da solução
s = np.zeros(k)                       ## solução corrente é o conjunto dos
                                      ## locais que serão instalacoes (medianas)
                                      ## um local é identificado por um inteiro
                                      ## de 0 à 50 (dado n=51)

s_star = np.zeros(k)                  ## melhor solução
fo = None                             ## funcao objetivo corrente
fo_star = None                        ## melhor funcao objetivo

## constroi matriz de distâncias
matriz_dist = constroi_matriz_distancias(n, locals)

In [274]:
# TODO: constroi uma solucao inicial aleatoria com k instalações (medianas)
def constroi_solucao_inicial(n, k, T, matriz_dist):
    s = np.zeros(k)  #vetor solução
    while(np.unique(s).shape[0] != k): # observe que números aleatórios podem
                                       # se repetir. np.unique(s) define um array
                                       # sem repetições para veriricação (K)
          s = np.random.randint(0, n, size=k)
    return s

# Escreve os locais definidos como medianas na solução
def escreve_instalacoes(s):
  print(s)
  return

# Calcula estruturas que irão associar  locais de instalação a locais usuários
# k: numero de instalações (medianas).
# s: solução atual com k posições para armazenar os locais (0..51) de instalações
def calcula_associacoes(s,n, k, matriz_dist):
    # Dado que, T=10 e K=5, teremos (n-k+1) usuários atendidos
    capacidade = np.zeros(k)  # controla o no. de locais atendidos por k instalações

    # vetor que associará usuários às instalaçoes
    usuarios_instalacao = np.zeros(n, dtype=np.int64)

    for j in np.arange(k):
          usuarios_instalacao[s[j]] = -1
    print(usuarios_instalacao)
    # Associa usuários à instalação mais próximas de acordo com sua capacidade máxima (T)
    for i in np.arange(n):
       if i not in s:
            dist_atendimentos = np.zeros(k)
            dist_atendimentos = [matriz_dist[i, j] for j in s]  # distâncias de i até as k medianas
            sorted_idx = np.argsort(dist_atendimentos)  ## veja como ordenar vetores no material
                        ## da disciplina: https://github.com/danielaterra/ifmg_topicos_SI/blob/main/python_NumpyArrays.ipynb
            inst_j = 0
            while (inst_j < k and capacidade[sorted_idx[inst_j]] >= T):
                    inst_j+=1
            if (inst_j >= k):
                usuarios_instalacao[i] = -1* sys.maxsize - 1 # sem atendimento para i
            else:
                j_mais_proximo = sorted_idx[inst_j]
                capacidade[j_mais_proximo]+=1
                usuarios_instalacao[i] = s[j_mais_proximo]

    return (s, usuarios_instalacao, capacidade)

# Calcula função objetivo
def calcula_fo(s, usuarios_instalacao, k, matriz_dist):
    fo = 0
    for i in np.arange(n):
        if (usuarios_instalacao[i] != -1):
             fo+= matriz_dist[i,usuarios_instalacao[i]]
    return fo

# Exibe facilitadores e usuários atendidos
def imprime_s(s, usuarios_instalacao, fo):
    print("Medianas em: ", s)
    print("Associações: \n", usuarios_instalacao)
    for j in s:
       usuarios = np.where(usuarios_instalacao == j) # retorna os indices em usuarios_instalacao que atendem à condição
       usuarios = np.array(usuarios).copy()
       print("Facilitador: ", j, "... usuarios: ",usuarios)
    print("Valor da solução: ", fo)

## Solução inicial

In [325]:
s = constroi_solucao_inicial(n, k, T, matriz_dist)

## Heurísticas de refinamento:  
Busca local: **trocas** entre facilitadores e usuários

* Para cada facilitador (ou mediana *j*)  na solução *s* atual, gere uma nova solução trocando *j* por um objeto (usuário) dentre os (n-k) usuários restantes.

* Cada uma das k*(n-k) soluções desta estrutura de vizinhança deve ser gerada. Veja alguns exemplos:
sendo n =[0,50[, k = 5 e s=(4, 40, 1, 17, 23), as primeiras e a últimas  soluções vizinhas serão:

s'(1)     = (0, 40, 1, 17, 23)

s'(2)     = (2, 40, 1, 17, 23)

..

s'(n-k-1) = (4, 40, 1, 17, 50)

s'(n-k)   = (4, 40, 1, 17, 51)

* Verifique e retorne a **melhor solução** encontrada nesta vizinhança

In [101]:
#TODO:

# Heurística de refinamento da solução 's':
def busca_local(n, s, usuarios_instalacao, matriz_dist):

    return  s_linha   #retorne apenas a melhor solução da estrutura de vizinhança!
